In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [78]:
from qiskit import QuantumCircuit, Aer, execute

#PARAMETERS

bpm = 4 #/4
measures = 60
beats = bpm * measures

key_shots = 50

#GET NOTE TYPES

type_conversion_dict = {
    '000': 4, #whole
    '001': 3, #dotted half
    '010': 2, #half
    '011': 1.5, #dotted quarter
    '100': 1, #quarter
    '101': 0.5, #eigth
    '110': 0.25, #sixteenth
    '111': 1 #quarter
}


type_circuit = QuantumCircuit(3, 3)

type_circuit.h([0,1,2])

type_circuit.measure([0,1,2], [0,1,2])

#Smallest possible outcome
t_shots_number = round(beats * 4)

backend = Aer.get_backend('qasm_simulator')
job_t = execute(type_circuit, backend, shots=t_shots_number, memory=True)
result_t = job_t.result()

# Get the list of outcomes in the order in which they occurred
type_outcomes = result_t.get_memory()

initial_types_list = []

for i in type_outcomes:
    note_type = type_conversion_dict[i]
    initial_types_list.append(note_type)

    
count = 0

types_list = []

for i in initial_types_list:
    if (count + i) <= beats:
        types_list.append(i)
        count += i
    else:
        pass
        
number_of_notes = len(types_list)


#GET NOTES


# Create a quantum circuit with 3 qubits and 3 classical bits
circuit = QuantumCircuit(3, 3)

# Apply Hadamard gates to all qubits to create superposition
circuit.h([0,1,2])

# Measure all qubits and record the results in the classical bits
circuit.measure([0,1,2], [0,1,2])

# Use the Qiskit simulator to run the circuit 1000 times with the memory option set to true
backend = Aer.get_backend('qasm_simulator')
job = execute(circuit, backend, shots=number_of_notes, memory=True)
result = job.result()

# Get the list of outcomes in the order in which they occurred
outcomes = result.get_memory()

notes_list = []

note_conversion_dict = {
    '000':'A',
    '001':'B',
    '010':'C',
    '011':'D',
    '100':'E',
    '101':'F',
    '110':'G',
    '111':'r'
}


for i in outcomes:
    note = note_conversion_dict[i]
    notes_list.append(note)


#GET NOTE "LEVEL"

level_circ = QuantumCircuit(1, 1)

level_circ.h(0)

level_circ.measure(0,0)

backend = Aer.get_backend('qasm_simulator')
job_l = execute(level_circ, backend, shots=t_shots_number, memory=True)
result_l = job_l.result()

# Get the list of outcomes in the order in which they occurred
level_outcomes = result_l.get_memory()

levels_list = []

levels = ["'","''"]

for i in level_outcomes:
    state = int(i)
    l = str(levels[state])
    levels_list.append(l)


#GET KEY SIGNATURE


#Part 1: Quantity

key_circuit = QuantumCircuit(3, 3)

key_circuit.h([0,1,2])

key_circuit.measure([0,1,2], [0,1,2])

backend = Aer.get_backend('qasm_simulator')
job_k = execute(key_circuit, backend, shots=10000)
result_k = job_k.result()

counts_dict = result_k.get_counts()

values = list(counts_dict.values()) #counts

keys = list(counts_dict.keys()) #measurements

largest_number = 0

final_key = ""

for i in range(len(values)):
    if values[i] > largest_number:
        largest_number = values[i]
        final_key = keys[i]

key_conversion_dict = {
    '000':0,
    '001':1,
    '010':2,
    '011':3,
    '100':4,
    '101':5,
    '110':6,
    '111':7
}

number_of_sf = key_conversion_dict[final_key]

#Part 2: Sharps or Flats

sf_circuit = QuantumCircuit(1, 1)

sf_circuit.h(0)

sf_circuit.measure(0,0)

backend = Aer.get_backend('qasm_simulator')
job_sf = execute(sf_circuit, backend, shots=10000)
result_sf = job_sf.result()

sf_dict = result_sf.get_counts()

values2 = list(sf_dict.values()) #counts

keys2 = list(sf_dict.keys()) #measurements

sharp_keys = ["C","G","D","A","E","B","Fis","Cis"]
flat_keys = ["C","F","Bes","Ees","Aes","Des","Ges","Ces"]

def apply_accidentals(is_sharp, num_accidentals, notes):
    new_notes = []
    if num_accidentals > 0:
        if is_sharp:
            sharps = ['F', 'C', 'G', 'D', 'A', 'E', 'D']
            accidental = "is"
            my_sharps = []
            for i in range(num_accidentals):
                my_sharps.append(sharps[i])
            for i in notes:
                if i.upper() in my_sharps:
                    n = i + "is"
                    new_notes.append(n)
                else:
                    new_notes.append(i)
                    
        else:
            flats = ['B', 'E', 'A', 'D', 'G', 'C', 'F']
            accidental = "es"
            my_flats = []
            for i in range(num_accidentals):
                my_flats.append(flats[i])
            for i in notes:
                if i.upper() in my_flats:
                    n = i + "es"
                    new_notes.append(n)
                else:
                    new_notes.append(i)
    else:
        new_notes = notes
        
    return new_notes

if values[0] > values[1]:
    #sharp
    key = sharp_keys[number_of_sf].lower()
    new_notes = apply_accidentals(True, number_of_sf, notes_list)
    
else:
    #flat
    key = flat_keys[number_of_sf].lower()
    new_notes = apply_accidentals(False, number_of_sf, notes_list)
    
#---------------------------------------------------------------
converted_types = []

for i in types_list:
    if i == 4:
        converted_types.append("1")
    elif i == 3:
        converted_types.append("2.")
    elif i == 2:
        converted_types.append("2")
    elif i == 1.5:
        converted_types.append("4.")
    elif i == 1:
        converted_types.append("8")
    elif i == 0.25:
        converted_types.append("16")
        

#---------------------------TESTING------------------------------#

print("  \key " + str(key) + " \major")

for i in range(len(converted_types)):
    note = new_notes[i].lower()
    if note == "r":
        final_line = note + converted_types[i]
    else:
        final_line = note + str(levels_list[i]) + converted_types[i]
    print(final_line)

  \key aes \major
g''1
des'8
ees'8
r8
des''1
c''8
des'2.
r16
des''4.
c''16
g'4.
bes'16
r4.
des''2.
aes'2
r2.
des'8
g''8
c''2.
r8
g'2
f'8
ees'2.
r16
des'4.
aes''8
bes''16
des'8
des'8
bes''16
c''1
c''2
f'8
c'8
r4.
des''2.
g''8
ees''8
r8
f'1
bes''16
r2.
f'1
f'8
f''8
g''8
c''4.
aes'4.
des''2.
des'2.
ees''4.
c'1
des'16
des''2.
f''2.
f''8
r2.
bes''8
ees''16
c''8
f'8
c'2.
des''1
des'2
f'4.
bes''8
aes'4.
des'2.
ees''2
aes''8
c''16
des''4.
aes'1
bes''1
aes'8
ees'2
g'1
g'16
des'1
bes''4.
f'2.
aes''16
ees'2.
r2.
r8
aes''16
c''8
g'1
des'4.
f''2
c'16
r1
aes'2
c''8
g'2.
ees'2
des''2
g'4.
aes'1
des''4.
g'8
des''2.
r8
aes''16
aes'8
bes'2.
r16
ees''2.
aes'4.
c'8
g''4.
f''2
f'4.
ees''2.
c''8
ees''2
g''4.
c'8
r2.
f'16
f''1
aes''4.
des'2.
bes''8
f'16
bes''2
